In [ ]:
pip install numpy --upgrade

In [ ]:
pip install mahotas

In [ ]:
import numpy as np
import mahotas

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
def save_video(fname, key_frames):
    count = 0

    cur_dir = fname[0:-4]
    print(cur_dir)
    os.mkdir(cur_dir)

    vidcap = cv2.VideoCapture(fname)
    success, img = vidcap.read()
    print('Read a new frame: ', success)

    while success:
        if count in key_frames:
            print(f'{count} in key frame')
            cv2.imwrite(cur_dir + "\\" + str(count) + ".jpg", img)
        success, img = vidcap.read()
        count += 1
    vidcap.release()
    
    return

def process_video(fname):
    """
        Uses Hu and Zernike moments

    """
    shape_features = []

    vidcap = cv2.VideoCapture(fname)
    success, img = vidcap.read()

    count = 0
    while success:
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(img2, 75, 200, cv2.THRESH_BINARY)[1]
        F = np.append(mahotas.features.zernike_moments(thresh, 250), cv2.HuMoments(cv2.moments(img2)).flatten())
        #print(F)
        shape_features.append(F)
        if(count%25 == 0):
            print("{count} images done".format(count=count))
        success, img = vidcap.read()
        count += 1
    
    vidcap.release()
        
    best_i = 0
    best_sc = -1

    for i in range(8,9):
        kmeans = KMeans(n_clusters=i, random_state=0).fit(shape_features)
        sc = silhouette_score(shape_features, kmeans.labels_)
        if(sc > best_sc):
            best_i = i
            best_sc = sc
        #print(i, silhouette_score(shape_features, kmeans.labels_))

    print(f'Best silhouette score was {best_sc} for k value of {best_i}')

    kmeans = KMeans(n_clusters=best_i, random_state=0).fit(shape_features)
    cents = kmeans.cluster_centers_

    print(len(shape_features), len(kmeans.labels_))

    min_dict = {}
    min_frame= {}
    for i in range(len(kmeans.labels_)):
        t = np.linalg.norm(shape_features[i]-cents[kmeans.labels_[i]])
        if kmeans.labels_[i] not in min_dict:
            min_dict[kmeans.labels_[i]] = t
            min_frame[kmeans.labels_[i]] = i
        else:
            if(t < min_dict[kmeans.labels_[i]]):
                min_dict[kmeans.labels_[i]] = t
                min_frame[kmeans.labels_[i]] = i

    print(min_frame)

    key_frames = list(min_frame.values())

    save_video(fname, key_frames)

In [ ]:
import os
from os import listdir

import cv2
for root, dirs, files in os.walk("F:\\UCF101_2\\UCF-101\\ApplyEyeMakeup", topdown=False):
    for name in files:
        print(os.path.join(root, name))
        process_video(os.path.join(root, name))